In [1]:
%autosave 0

Autosave disabled


In [2]:
# !pip install grpcio tensorflow-serving-api==2.17.0

In [3]:
# !pip install keras-image-helper

In [4]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2024-12-16 22:44:27.244714: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-16 22:44:27.249245: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-16 22:44:27.264903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 22:44:27.292498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 22:44:27.299764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 22:44:27.311415: I tensorflow/core/platform/cpu_feature_gu

In [5]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [6]:
from keras_image_helper import create_preprocessor

In [7]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [8]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [9]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [16]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['inputs'].CopyFrom(np_to_protobuf(X))

In [ ]:
pb_request

In [18]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.FAILED_PRECONDITION
	details = "Could not find variable block12_sepconv3_bn/gamma. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status error message=Resource localhost/block12_sepconv3_bn/gamma/N10tensorflow3VarE does not exist.
	 [[{{function_node __inference_serving_default_3416}}{{node functional_14_1/xception_1/block12_sepconv3_bn_1/Cast_2/ReadVariableOp}}]]"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:8500 {created_time:"2024-12-16T22:48:21.605432543+00:00", grpc_status:9, grpc_message:"Could not find variable block12_sepconv3_bn/gamma. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status error message=Resource localhost/block12_sepconv3_bn/gamma/N10tensorflow3VarE does not exist.

In [ ]:
preds = pb_response.outputs['output_0'].float_val

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [ ]:
dict(zip(classes, preds))